In [1]:
import torch
from torch.nn import Sequential, Linear, ReLU, Sigmoid, Tanh, Dropout, Upsample
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import BatchNorm
import numpy as np
from torch_geometric.data import Data
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torch.distributions import normal, kl

from tqdm import tqdm

from sklearn.model_selection import KFold
import pandas as pd
from MatrixVectorizer import MatrixVectorizer


In [2]:
# set global variables
N_SUBJECTS = 167

N_LR_NODES = 160

N_HR_NODES = 268

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

N_LR_NODES_F = int(N_LR_NODES * (N_LR_NODES-1) / 2)
N_HR_NODES_F = int(N_HR_NODES * (N_HR_NODES-1) / 2)

# Model Layers

In [3]:
class SheafConvLayer(nn.Module):
    def __init__(self, n_nodes, d, f_in, f_out=None):
        super().__init__()
        self.d = d
        self.n_nodes = n_nodes
        self.f_in = f_in
        self.f_out = f_out
        # random init weight matrices
        if f_out is None:
            f_out = f_in 
        self.weight1 = nn.Parameter(torch.randn((d, d), device=DEVICE))
        self.weight2 = nn.Parameter(torch.randn((f_in, f_out), device=DEVICE))
        self.edge_weights = nn.Parameter(torch.randn((n_nodes, n_nodes, d, 2*d), device=DEVICE))


    def forward(self, X, adj):
        kron_prod = torch.kron(torch.eye(self.n_nodes).to(DEVICE), self.weight1)
        L = self.sheaf_laplacian(X, adj)
        if self.f_out is None:
            return X - F.elu(L @ kron_prod @ X @ self.weight2), L
        else:
            return F.elu(L @ kron_prod @ X @ self.weight2), L


    def sheaf_laplacian(self, X, adj, epsilon=1e-6):
        X_reshaped = X.reshape(self.n_nodes, self.d, -1)
        idx_pairs = torch.cartesian_prod(torch.arange(self.n_nodes), torch.arange(self.n_nodes))
        all_stacked_features = X_reshaped[idx_pairs].reshape(self.n_nodes, self.n_nodes, 2*self.d, -1).to(DEVICE)
        lin_trans = F.elu(torch.matmul(self.edge_weights, all_stacked_features))
        inner_transpose = torch.transpose(lin_trans, -1, -2)
        L_v = -1 * torch.matmul(lin_trans, torch.transpose(inner_transpose, 0, 1))
        # row_cond = torch.isclose(torch.sum(adj, dim=1), torch.zeros_like(torch.sum(adj, dim=1)))
        # col_cond = torch.isclose(torch.sum(adj, dim=0), torch.zeros_like(torch.sum(adj, dim=0)))
        adj_row_weights = adj / (torch.sum(adj, dim=1)[:, None] + epsilon)
        adj_col_weights = adj / (torch.sum(adj, dim=0)[:, None] + epsilon)
        # adj_col_weights = torch.where(col_cond[None, :], 0., adj / torch.sum(adj, dim=0)[None, :])
        adj_weights = torch.maximum(adj_row_weights * adj_col_weights, torch.zeros_like(adj_row_weights))

        adj_diag_weights = adj_row_weights ** 2
        diag_blocks = torch.sum(adj_diag_weights[:, :, None, None] * torch.matmul(lin_trans, inner_transpose), dim=1)
        L_v[range(self.n_nodes), range(self.n_nodes)] = diag_blocks
        return L_v.reshape(-1, self.n_nodes * self.d)
        ### NOTE IGNORE MATRIX NORMALISATION FOR NOW #####
        # inv_root_diag_blocks = torch.pow(diag_blocks+epsilon, -1/2)
        # normalise_mat = torch.block_diag(*inv_root_diag_blocks)

        # return normalise_mat @ L_v.view(-1, self.n_nodes * self.d) @ normalise_mat
        ################################################

In [4]:
class AdjacencyDimChanger(nn.Module):
    def __init__(self, new_n, old_n, old_f, d):
        super().__init__()
        self.new_n = new_n
        self.old_n = old_n
        self.d = d
        self.sheafconv = SheafConvLayer(old_n, d, old_f, new_n)
        self.layernorm = nn.LayerNorm([d, old_n]).to(DEVICE)

    def forward(self, X, adj):

        adj = adj - torch.diag_embed(torch.diagonal(adj, 0)).to(DEVICE) + torch.eye(adj.shape[0]).to(DEVICE)  # add self connections
        x, L = self.sheafconv(X, adj)

        x = x.reshape(self.old_n, self.d, self.new_n)
        x = torch.transpose(x, 0, -1)
        x = self.layernorm(x)
        
        x_mean = x.mean(dim=-1)

        L_mean = L.reshape(self.old_n, self.old_n, self.d, self.d).max(dim=0)[0].mean(dim=0) # aggregate by eigenvalues of each n by n mat?
        adj_new = torch.matmul(x_mean, L_mean)
        adj_new = torch.matmul(adj_new, x_mean.T)
        adj_new_T = torch.t(adj_new)
        adj_new = F.tanh(F.relu(((adj_new + adj_new_T) / 2))) # becomes a new f by new f adj1

        return x.reshape(self.new_n*self.d, -1), adj_new

In [5]:
class AdjacencyChangerUp(nn.Module):

    def __init__(self, d, f_in):
        super().__init__()
        self.d = d

        self.adjdim_changer1 = AdjacencyDimChanger(216, N_LR_NODES, f_in, d)
        self.adjdim_changer2 = AdjacencyDimChanger(256, 216, N_LR_NODES, d)
        self.adjdim_changer3 = AdjacencyDimChanger(N_HR_NODES, 256, 216, d)

        # self.adjdim_changer1 = AdjacencyDimChanger(256, N_LR_NODES, f_in, d)
        # self.adjdim_changer2 = AdjacencyDimChanger(N_HR_NODES, 256, N_LR_NODES, d)

        
    def forward(self, X, adj):
        x1, adj1 = self.adjdim_changer1(X, adj)
        x2, adj2 = self.adjdim_changer2(x1, adj1)
        # return [adj, adj1, adj2]

        x3, adj3 = self.adjdim_changer3(x2, adj2)
        return [adj, adj1, adj2, adj3]
        

In [6]:
class AdjacencyChangerDown(nn.Module):

    def __init__(self, d, f_in):
        super().__init__()
        self.d = d

        self.adjdim_changer1 = AdjacencyDimChanger(256, N_HR_NODES, f_in, d)
        self.adjdim_changer2  = AdjacencyDimChanger(216, 256, N_HR_NODES, d)
        self.adjdim_changer3 = AdjacencyDimChanger(N_LR_NODES, 216, 256, d)

        # self.adjdim_changer1 = AdjacencyDimChanger(256, N_HR_NODES, f_in, d)
        # self.adjdim_changer2  = AdjacencyDimChanger(N_LR_NODES, 256, N_HR_NODES, d)

        
    def forward(self, X, adj):
        x1, adj1 = self.adjdim_changer1(X, adj)
        x2, adj2 = self.adjdim_changer2(x1, adj1)
        # return [adj, adj1, adj2]


        x3, adj3 = self.adjdim_changer3(x2, adj2)
        return [adj, adj1, adj2, adj3]

In [7]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False

def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True

In [7]:
import numpy as np
import networkx as nx

def eigen_centrality(data):
    # ROI is the number of brain regions (i.e.,35 in our case)
    ROI = 160

    topology_eigen = []

    G = nx.from_numpy_array(np.absolute(data))
    U = G.to_undirected()

    # A = to_2d(data)
    np.fill_diagonal(data, 0)

    # create a graph frL2
    # # compute egeinvector centrality and transform the output to vector
    ec = nx.eigenvector_centrality_numpy(U)
    
    eigenvector_centrality = np.array([ec[g] for g in U])



    topology_eigen.append(eigenvector_centrality)  # 2

    return topology_eigen

def pearson_coor(input, target, epsilon=1e-7):
    vx = input - torch.mean(input, dim=(1, 2))[:, None, None]
    vy = target - torch.mean(target, dim=(1, 2))[:, None, None]
    cost = torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)+epsilon) * torch.sqrt(torch.sum(vy ** 2)+epsilon)+epsilon)
    return cost

def GT_loss(target, predicted):

    # l1_loss
    l1_loss = torch.nn.L1Loss()
    # loss_pix2pix = l1_loss(target, predicted)

    # topological_loss
    target_n = target.detach().cpu().clone().numpy()
    predicted_n = predicted.detach().cpu().clone().numpy()
    torch.cuda.empty_cache()

    topo_loss = []
    

    for i in range(len(target_n)):

        cur_target = target_n[i]
        cur_predicted = predicted_n[i]

        target_t = eigen_centrality(cur_target)
        real_topology = torch.tensor(target_t[0])
        predicted_t = eigen_centrality(cur_predicted)
        fake_topology = torch.tensor(predicted_t[0])
        topo_loss.append(l1_loss(real_topology, fake_topology))

    topo_loss = torch.sum(torch.stack(topo_loss))

    pc_loss = pearson_coor(target, predicted).to(DEVICE)
    torch.cuda.empty_cache()

    # G_loss = loss_pix2pix + (1 - pc_loss) + topo_loss
    G_loss = (1 - pc_loss) + topo_loss


    return G_loss

In [8]:
def loss_calc(adj_ls, opp_adj_ls):
    total_loss = torch.Tensor([0]).to(DEVICE)
    mse_loss_fn = torch.nn.MSELoss()
    n = len(adj_ls)

    mse_loss = torch.Tensor([0]).to(DEVICE)
    for i, (adj, opp_adj) in enumerate(zip(adj_ls[::-1], opp_adj_ls)):
        weight = 2 * (i + 1) / (n * (1 + n))
        mse_loss = mse_loss + mse_loss_fn(adj, opp_adj) * weight

    gt_loss = torch.Tensor([0]).to(DEVICE)
    for i, (adj, opp_adj) in enumerate(zip(adj_ls[::-1], opp_adj_ls)):

        ### NOTE TEMPORARY MEASURE BECAUSE THEY TAKE IN (BATCHSIZE, xx, xx) shape ####
        temp_adj = adj.reshape(1, *adj.shape)
        temp_opp_adj = opp_adj.reshape(1, *opp_adj.shape)
        ##########################################################
        gt_loss = gt_loss + GT_loss(temp_adj, temp_opp_adj)

    gt_loss = gt_loss / n
        
    return mse_loss + gt_loss

# Training

In [8]:
from data_preparation import load_data_tensor

lr_train, lr_test, hr_train = load_data_tensor("dgl-icl")

In [11]:
lr_X_dim1 = torch.load('model_autoencoder/encode_lr_1.pt')
lr_X_dim2 = torch.load('model_autoencoder/encode_lr_2.pt')
hr_X_dim1 = torch.load('model_autoencoder/encode_hr_1.pt')
hr_X_dim2 = torch.load('model_autoencoder/encode_hr_2.pt')


lr_X_all = torch.empty((167, 320, 32))
for i in range(len(lr_X_dim1)):
    a, b = lr_X_dim1[i], lr_X_dim2[i]
    lr_X_all[i] = torch.cat([a, b], dim=-1).view(-1, a.shape[-1])

hr_X_all = torch.empty((167, 536, 32))
for i in range(len(hr_X_dim1)):
    a, b = hr_X_dim1[i], hr_X_dim2[i]
    hr_X_all[i] = torch.cat([a, b], dim=-1).view(-1, a.shape[-1])

In [12]:
trainloader = DataLoader(list(zip(lr_X_all, lr_train, hr_X_all, hr_train)), shuffle=True, batch_size=8)


up_changer = AdjacencyChangerUp(d=2,f_in=32).to(DEVICE)
down_changer = AdjacencyChangerDown(d=2,f_in=32).to(DEVICE)

up_optimizer = torch.optim.AdamW(up_changer.parameters(), lr=0.001, betas=(0.5, 0.999))
down_optimizer = torch.optim.AdamW(down_changer.parameters(), lr=0.001, betas=(0.5, 0.999))

sum(p.numel() for model in [up_changer, down_changer] for p in model.parameters())


2792776

In [13]:
def train(epochs, up_changer, down_changer, trainloader, up_optimizer, down_optimizer):

    up_changer.train()
    down_changer.train()
    for epoch in range(epochs):

        up_losses = []
        down_losses = []

        for X_lr, adj_lr, X_hr, adj_hr in tqdm(trainloader):

            freeze_model(up_changer)
            unfreeze_model(down_changer)
        
            down_optimizer.zero_grad()
            up_optimizer.zero_grad()

            down_batch_loss = []

            for i in range(len(X_lr)):

                up_adj_ls = up_changer(X_lr[i].to(DEVICE), adj_lr[i].to(DEVICE))
                torch.cuda.empty_cache()
                down_adj_ls = down_changer(X_hr[i].to(DEVICE), adj_hr[i].to(DEVICE))
                torch.cuda.empty_cache()

                down_batch_loss.append(loss_calc(down_adj_ls, up_adj_ls))

            down_loss = torch.mean(torch.stack(down_batch_loss))
            down_loss.backward()
            down_optimizer.step()

            print(f'Down Loss = {down_loss.detach().item()}')
            down_losses.append(down_loss.detach().item())
            del down_loss
            del down_batch_loss
            torch.cuda.empty_cache()

            unfreeze_model(up_changer)
            freeze_model(down_changer)
        
            down_optimizer.zero_grad()
            up_optimizer.zero_grad()

            up_batch_loss = []


            for i in range(len(X_lr)):

                up_adj_ls = up_changer(X_lr[i].to(DEVICE), adj_lr[i].to(DEVICE))
                torch.cuda.empty_cache()
                down_adj_ls = down_changer(X_hr[i].to(DEVICE), adj_hr[i].to(DEVICE))
                torch.cuda.empty_cache()
            
                up_batch_loss.append(loss_calc(up_adj_ls, down_adj_ls))

            up_loss = torch.mean(torch.stack(up_batch_loss))
            up_loss.backward()
            up_optimizer.step()

            print(f'Up Loss = {up_loss.detach().item()}')
            up_losses.append(up_loss.detach().item())
            del up_loss
            del up_batch_loss
            torch.cuda.empty_cache()



        epoch_up_loss = np.mean(up_losses)
        epoch_down_loss = np.mean(down_losses)

        print(f'epoch {epoch}: down loss = {epoch_down_loss}, up loss = {epoch_up_loss}')

    return up_changer, down_changer


In [14]:
from evaluation_fn import evaluate_predictions


def validation(up_changer, val_X_lr, val_adj_lr, val_adj_hr):
    print('begin validation')
    up_changer.eval()

    all_predictions = torch.empty((len(val_X_lr), N_HR_NODES, N_HR_NODES), requires_grad=False).cpu()

    for i in tqdm(range(len(val_X_lr))):
        all_predictions[i] = up_changer(val_X_lr[i].to(DEVICE), val_adj_lr[i].to(DEVICE))[-1].detach()
        torch.cuda.empty_cache()

    return evaluate_predictions(all_predictions, val_adj_hr)


In [15]:
def cross_validate(model, epochs, n_fold, X_lr, adj_lr, X_hr, adj_hr, d=2, f=32):
    kf = KFold(n_fold, shuffle=True, random_state=99)
    runs_results = []
    for train_idx, val_idx in kf.split(X_lr):
        train_X_lr, val_X_lr = X_lr[train_idx], X_lr[val_idx]
        train_adj_lr, val_adj_lr = adj_lr[train_idx], adj_lr[val_idx]
        train_X_hr = X_hr[train_idx]
        train_adj_hr, val_adj_hr = adj_hr[train_idx], adj_hr[val_idx]

        trainloader = DataLoader(list(zip(train_X_lr, train_adj_lr, train_X_hr, train_adj_hr)), shuffle=True, batch_size=8)

        up_changer = AdjacencyChangerUp(d=d,f_in=f).to(DEVICE)
        down_changer = AdjacencyChangerDown(d=d,f_in=f).to(DEVICE)

        up_optimizer = torch.optim.AdamW(up_changer.parameters(), lr=0.001, betas=(0.5, 0.999))
        down_optimizer = torch.optim.AdamW(down_changer.parameters(), lr=0.001, betas=(0.5, 0.999))    
        
        up_changer, down_changer = train(epochs, up_changer, down_changer, trainloader, up_optimizer, down_optimizer)
        model.append(up_changer)
        val_metrics = validation(up_changer, val_X_lr, val_adj_lr, val_adj_hr)
        runs_results.append(val_metrics)

    return runs_results


In [ ]:
models=[]
cv_results = cross_validate(models, 10, 3, lr_X_all, lr_train, hr_X_all, hr_train)

In [16]:
# train on entire dataset
up_changer, down_changer = train(20, up_changer, down_changer, trainloader, up_optimizer, down_optimizer)

  0%|          | 0/21 [00:00<?, ?it/s]

Down Loss = 2.2756166458129883


  5%|▍         | 1/21 [00:24<08:11, 24.59s/it]

Up Loss = 2.226686716079712
Down Loss = 2.2351632118225098


 10%|▉         | 2/21 [00:48<07:42, 24.34s/it]

...

Up Loss = 0.861221432685852
Down Loss = 0.9128678441047668


 95%|█████████▌| 20/21 [13:02<00:39, 39.27s/it]

Up Loss = 0.8962550163269043
Down Loss = 0.9261093139648438


100%|██████████| 21/21 [13:36<00:00, 38.87s/it]

Up Loss = 0.9044310450553894
epoch 19: down loss = 0.8977801743007842, up loss = 0.8775011329423814


In [17]:
torch.save(up_changer.state_dict(), 'tim_files/up_changer.pth')

In [9]:
up_changer = AdjacencyChangerUp(d=2,f_in=32).to(DEVICE)
up_changer.load_state_dict(torch.load('tim_files/up_changer.pth'))

<All keys matched successfully>

In [10]:
lr_X_test_dim1 = torch.load('model_autoencoder/test_hr_1.pt')
lr_X_test_dim2 = torch.load('model_autoencoder/test_hr_2.pt')


lr_test_X_all = torch.empty((112, 320, 32))
for i in range(len(lr_test_X_all)):
    a, b = lr_X_test_dim1[i], lr_X_test_dim2[i]
    lr_test_X_all[i] = torch.cat([a, b], dim=-1).view(-1, a.shape[-1])


In [11]:
pred = torch.empty((112, 268, 268))
up_changer.eval()
for i in range(len(lr_test)):
    pred[i] = up_changer(lr_test_X_all[i].to(DEVICE), lr_test[i].to(DEVICE))[-1].detach()